# Homework 1: SoftwareIntro
The purpose of this homework is to guide you through opening the programs you've downloaded through Neurodebian and getting familiar with some of their basic functions.


## Preparation 

### Load python package
The following python package is necessary to view the nifti images that we will be working with in this tutorial.

In [ ]:
from niwidgets import NiftiWidget

### Download the data
We need one more python package to download publically available data for viewing, and we will download one subject for playing with.

In [ ]:
from nilearn import datasets
haxby_dataset = datasets.fetch_haxby()
func_filename = haxby_dataset.func[0]
anat_filename = haxby_dataset.anat[0]

# MV
# note download of the last file takes a really long time with a slow-ish connection
# consider getting most if not all of data needed for labs loaded in to the machine before they download it?


### Preview the data in jupyter

In [ ]:
anat_widget = NiftiWidget(anat_filename)
anat_widget.nifti_plotter()

In [ ]:
import nilearn.plotting as nip
anat_widget.nifti_plotter(plotting_func=nip.plot_epi, display_mode=['ortho', 'x', 'y', 'z', 'yx', 'xz', 'yz'])

## Problem 1

In [ ]:
anat_filename

In [ ]:
# modify text below to fit with the image they will open and the viewers they will see
# change problem one to be reporting the histogram from fslview
# for submitting, we would teach students how to take a screenshot in their vbox and import to the notebook

# make problem 2 a demo of nilearn plotting

%%bash
fslview /home/brain/nilearn_data/haxby2001/subj2/anat.nii.gz &

This is a high-resolution image of someone's brain using [T1 contrast](http://www.med.harvard.edu/aanlib/basicsMR.html), and the image format is [nifti](https://brainder.org/2012/09/23/the-nifti-file-format/). The image will be opened in 3 orthogonal views simultaneously; coronal, sagittal and axial views. Notice the labeling: L, R, A, P, S, I (Left, Right, Anterior=front, Posterior=back, Superior=top, Inferior=bottom). Scroll through the brain and get familiar with the brain. Note in MRI we cannot image at the level of single neurons, so the anatomy we focus on is from a "macroscopic" view. Here is an example of what you should see, with some annotations on tools used in the Homework. Note shown here is the "standard brain" without skull. Your image should have skull.

![fslview_annot](files/img/fslview_annotated.png "erh mer gerd herver terxt")

In FSLview, one quick way to get information about images or to change their color scale is to use the widget opened by the little blue “i” button that is to the bottom left of the “Overlay settings” box (see figure above). Click this button now, and you should see something like this:

![fslview_info_box](files/img/example_info_dialog_fslview.png "that is a nice dialog box")

Where:
- **Filename:** the path of the image you’re looking at, e.g., where is it on your computer and what is the name of the image?
- **Voxels:** the size of the 3D “grid” of voxels that hold your image, more details on this later
- **Dimensions:** the size of each voxel
- **Volumes:** how many 3D volumes, or brains, you have in this image file; structural images typically have 1 volume, whereas functional images have a series of volumes acquired over time
- The other item we care about on this menu is the “Lookup table options” button. This is where you can change the color scale of the image. Play around with this, and see what you like! We usually use this for statistical maps, but it can also be helpful when comparing two images overlaid on top of one another. We will do this later!


**For now, take a screen shot of your T1 image in some color scale other than grayscale and insert this into your word document homework for problem #1**






## Problem 2

Another useful tool when looking at images is to look at the image histogram. This is a histogram of all the intensity values that make up the image. Looking at these is helpful to see what intensities are generally “brain” versus background and to get comfortable with the idea that images are a collection of voxels with different intensity values. To illustrate this, below is a figure from the Handbook of Functional MRI Data Analysis:

![poldrack_image](files/img/poldrack_images_are_intensities.png "'nother hover")

Remember, the intensity values mean something about what tissue we think we’re seeing (based on the physics of MRI). It is also helpful to remember that the basis of most image processing tools is doing signal processing and statistics on these intensity values. Find this in the FSLview GUI to track the intensity value of each voxel your cursor is on.

To view the histogram for the entire T1 image, open in FSLview, go to the “Tools” menu and click on “Image Histogram.” Can you tell the approximate range of values that make up “brain” versus background in this image?

**Provide a screen shot of the histogram and report this range for your answer to problem #2.**


## Skip Problem 3?


## Problem 4

The black lines represent the time series data from the grid of voxels within the green square cursor at the center of the cross-hair in the brain viewer window. To review, fMRI data are a set of intensity values for 3D images of the brain that are collected over time. For instance, imagine the structural image above (and associated intensity values you see in the magnified matrix) now collected multiple times over the course of 5 minutes. The result is each voxel has not only one intensity value, rather each voxel now has a timeseries of intensity values. In functional MRI these intensity values reflect the BOLD response, which we'll talk about more next week. For now, it may be helpful to see this idea visualized again from the Aguirre 2014 paper we read:

MV: below is too complex for the first lab; our goal for this one is to have them open a functional image in a viewer and play around with seeing the resolution and udnerstanding what a timeseries looks like from different voxels. This could be problem 3. I like afni viewer for this because it introduces them to a common viewer and the grid is nice for learning how the timeseries changes as you move around the brain.

Then have problem 4 be a simple coding challenge to get warmed up on basic commands we will use a lot. May include a problem for setting up their github configuration to their username on their virtual machine.

In [ ]:
%%bash
afni /home/brain/nilearn_data/haxby2001/subj2/bold.nii.gz &

In [ ]:
# mv suggests we hold this after we talk about what masks are 

import nibabel as nib
from nilearn import input_data
# load the face mask from the haxby dataset into python
face_mask_img = nib.load(haxby_dataset.mask_face[0])
# load the house mask from the haxby dataset into python
house_mask_img = nib.load(haxby_dataset.mask_house[0])
# concatonate the masks to make 4D image (each 3D image has one mask)
haxby_maps = nib.funcs.concat_images([face_mask_img, house_mask_img])

# nilearn utility to apply the masks to a dataset
masker = input_data.NiftiMapsMasker(
    haxby_maps, resampling_target="data", detrend=True, memory='nilearn_cache', memory_level=10, verbose=2)
# get the timeseries 
time_series = masker.fit_transform(func_filename)

In [ ]:
%matplotlib inline
haxby_dataset
import matplotlib.pyplot as plt

# plot the two time series
face_plot, = plt.plot(time_series[:,0], label='face')
house_plot, = plt.plot(time_series[:,1], label='house')
plt.legend([face_plot, house_plot], ['face', 'house'])
plt.show()
